In [29]:
import pandas as pd
from rdkit import Chem
from mordred import Calculator, descriptors
data_pfp=pd.read_csv('Fiehn_PFP.csv')
data_csh=pd.read_csv('Fiehn_CSH.csv')
data_pfp.head()
data_csh.head()

,compound_name,retention_time,smiles,split_index
0,1_TG 14:0-13:0-14:0-d5_ISTD,9.211,CCCCCCCCCCCCCC(=O)OCC(COC(=O)CCCCCCCCCCCCC)OC(=O)CCCCCCCCCCCC,1
1,1_TG 14:0-15:1-14:0-d5_ISTD,9.235,CCCC/C=C\CCCCCCCCC(=O)OC(COC(=O)CCCCCCCCCCCCC)COC(=O)CCCCCCCCCCCCC,1
2,1_TG 14:0-17:1-14:0-d5_ISTD,9.742,CCCCCC/C=C\CCCCCCCCC(=O)OC(COC(=O)CCCCCCCCCCCCC)COC(=O)CCCCCCCCCCCCC,2
3,1_TG 16:0-15:1-16:0-d5_ISTD,10.223,CCCC/C=C\CCCCCCCCC(=O)OC(COC(=O)CCCCCCCCCCCCCCC)COC(=O)CCCCCCCCCCCCCCC,1
4,1_TG 16:0-17:1-16:0-d5_ISTD,10.671,CCCCCC/C=C\CCCCCCCCC(=O)OC(COC(=O)CCCCCCCCCCCCCCC)COC(=O)CCCCCCCCCCCCCCC,1


In [33]:
calc = Calculator(descriptors, ignore_3D=True)
mols = [Chem.MolFromSmiles(smi) for smi in data_pfp['smiles']]
df = calc.pandas(mols)
df.head()

  2%|▏         | 33/1933 [00:04<03:18,  9.58it/s] 

C:\Users\halle\miniconda3\envs\my-rdkit-env\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  2%|▏         | 38/1933 [00:05<03:19,  9.51it/s]

C:\Users\halle\miniconda3\envs\my-rdkit-env\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  7%|▋         | 127/1933 [00:14<02:49, 10.66it/s]

...

C:\Users\halle\miniconda3\envs\my-rdkit-env\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 62%|██████▏   | 1208/1933 [02:04<00:55, 13.17it/s]

C:\Users\halle\miniconda3\envs\my-rdkit-env\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 1933/1933 [03:14<00:00,  9.92it/s]


,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,53.346601,39.877147,1,0,84.547542,2.735936,5.454882,84.547542,1.261904,5.155252,...,11.669322,122.404157,954.482430,6.769379,20233,154,388.0,494.0,27.479167,14.104167
1,15.361848,13.421903,0,0,24.365139,2.447054,4.894108,24.365139,1.218257,3.914319,...,9.989344,53.842497,278.115424,7.318827,763,32,104.0,121.0,8.479167,4.333333
2,17.106680,15.468742,0,0,25.234080,2.631977,5.263954,25.234080,1.147004,4.022669,...,10.540250,57.336429,310.214409,5.965662,888,45,122.0,150.0,10.993056,4.555556
3,42.885836,33.565709,0,0,66.749110,2.728079,5.427230,66.749110,1.236095,4.934765,...,11.427030,108.258079,770.481642,6.213562,10668,118,310.0,389.0,22.951389,11.256944
4,25.938492,20.372524,0,0,43.505650,2.437816,4.875633,43.505650,1.279578,4.433325,...,10.367316,70.107172,480.163162,7.744567,3873,55,172.0,200.0,12.833333,7.611111


In [10]:
split = data_pfp['split_index']
rt = data_pfp['retention_time']
df['split_index'] = split
df['retention_time'] = rt
df.head()

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,split_index,retention_time
0,53.346601,39.877147,1,0,84.547542,2.735936,5.454882,84.547542,1.261904,5.155252,...,954.482430,6.769379,20233,154,388.0,494.0,27.479167,14.104167,2,4.698
1,15.361848,13.421903,0,0,24.365139,2.447054,4.894108,24.365139,1.218257,3.914319,...,278.115424,7.318827,763,32,104.0,121.0,8.479167,4.333333,1,2.801
2,17.106680,15.468742,0,0,25.234080,2.631977,5.263954,25.234080,1.147004,4.022669,...,310.214409,5.965662,888,45,122.0,150.0,10.993056,4.555556,2,3.907
3,42.885836,33.565709,0,0,66.749110,2.728079,5.427230,66.749110,1.236095,4.934765,...,770.481642,6.213562,10668,118,310.0,389.0,22.951389,11.256944,1,4.410
4,25.938492,20.372524,0,0,43.505650,2.437816,4.875633,43.505650,1.279578,4.433325,...,480.163162,7.744567,3873,55,172.0,200.0,12.833333,7.611111,1,3.008


In [11]:
df_train=df.loc[df['split_index'] == 1]
df_train.head()
df_test=df.loc[df['split_index'] == 2]
df_test.head()

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,split_index,retention_time
0,53.346601,39.877147,1,0,84.547542,2.735936,5.454882,84.547542,1.261904,5.155252,...,954.482430,6.769379,20233,154,388.0,494.0,27.479167,14.104167,2,4.698
2,17.106680,15.468742,0,0,25.234080,2.631977,5.263954,25.234080,1.147004,4.022669,...,310.214409,5.965662,888,45,122.0,150.0,10.993056,4.555556,2,3.907
7,24.570965,19.968537,1,0,38.781891,2.591037,5.182075,38.781891,1.251029,4.376781,...,422.172939,7.406543,2234,59,172.0,209.0,11.895833,6.555556,2,5.928
18,15.613368,12.228372,0,0,26.115489,2.448426,4.896852,26.115489,1.305774,3.935766,...,268.073559,8.377299,821,33,106.0,126.0,6.277778,4.444444,2,4.585
32,28.155878,20.267691,0,0,44.141810,2.538201,5.076401,44.141810,1.261195,4.509461,...,476.183503,7.558468,3732,67,200.0,243.0,12.708333,7.263889,2,6.062


In [12]:
from autogluon.tabular import TabularDataset, TabularPredictor
label='retention_time'
print("Summary of rt variable: \n", df_train[label].describe())

Summary of rt variable: 
 count    1391.000000
mean        3.848717
std         1.805281
min         0.212000
25%         2.315000
50%         4.055000
75%         5.482000
max         8.560000
Name: retention_time, dtype: float64


In [13]:
save_path ='rt_predict_models_pfp'
predictor = TabularPredictor(label=label, path=save_path).fit(df_train)

Beginning AutoGluon training ...
AutoGluon will save models to "rt_predict_models\"
AutoGluon Version:  0.4.0
Python Version:     3.9.12
Operating System:   Windows
Train Data Rows:    1391
Train Data Columns: 1614
Label Column: retention_time
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (8.56, 0.212, 3.84872, 1.80528)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    7423.76 MB
	Train Data (Original)  Memory Usage: 33.08 MB (0.4% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify

In [22]:
y_test = df_test[label]
test_data_nolab = df_test.drop(columns=[label])
test_data_nolab.head()

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,split_index
0,53.346601,39.877147,1,0,84.547542,2.735936,5.454882,84.547542,1.261904,5.155252,...,122.404157,954.482430,6.769379,20233,154,388.0,494.0,27.479167,14.104167,2
2,17.106680,15.468742,0,0,25.234080,2.631977,5.263954,25.234080,1.147004,4.022669,...,57.336429,310.214409,5.965662,888,45,122.0,150.0,10.993056,4.555556,2
7,24.570965,19.968537,1,0,38.781891,2.591037,5.182075,38.781891,1.251029,4.376781,...,67.638644,422.172939,7.406543,2234,59,172.0,209.0,11.895833,6.555556,2
18,15.613368,12.228372,0,0,26.115489,2.448426,4.896852,26.115489,1.305774,3.935766,...,53.909997,268.073559,8.377299,821,33,106.0,126.0,6.277778,4.444444,2
32,28.155878,20.267691,0,0,44.141810,2.538201,5.076401,44.141810,1.261195,4.509461,...,72.423241,476.183503,7.558468,3732,67,200.0,243.0,12.708333,7.263889,2


In [24]:
predictor = TabularPredictor.load(save_path)
y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

C:\Users\halle\miniconda3\envs\my-rdkit-env\lib\site-packages\autogluon\features\generators\datetime.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_datetime[datetime_feature] = pd.to_datetime(X[datetime_feature], errors='coerce').fillna(self._fillna_map[datetime_feature])
C:\Users\halle\miniconda3\envs\my-rdkit-env\lib\site-packages\autogluon\features\generators\datetime.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_datetime[datetime_feature + '.' + feature] = getattr(X_datetime[datetime_feature].dt, feature).

Predictions:  
 0       4.664629
2       4.273823
7       5.866498
18      4.630483
32      5.867785
          ...   
1498    2.233401
1509    4.220539
1513    3.273458
1534    6.150897
1541    0.291267
Name: retention_time, Length: 155, dtype: float32
<class 'dict'>


In [16]:
df_modelpf = predictor.leaderboard(df_test , silent=True)

C:\Users\halle\miniconda3\envs\my-rdkit-env\lib\site-packages\autogluon\features\generators\datetime.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_datetime[datetime_feature] = pd.to_datetime(X[datetime_feature], errors='coerce').fillna(self._fillna_map[datetime_feature])
C:\Users\halle\miniconda3\envs\my-rdkit-env\lib\site-packages\autogluon\features\generators\datetime.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_datetime[datetime_feature + '.' + feature] = getattr(X_datetime[datetime_feature].dt, feature).

<class 'pandas.core.frame.DataFrame'>


In [40]:
df_modelpf.to_csv('model_performance_pfp', index=False)

In [34]:
calc2 = Calculator(descriptors, ignore_3D=True)
mols2 = [Chem.MolFromSmiles(smi) for smi in data_csh['smiles']]
df2 = calc2.pandas(mols2)
df2.head()

  0%|          | 1/259 [00:02<09:53,  2.30s/it]

C:\Users\halle\miniconda3\envs\my-rdkit-env\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  1%|          | 3/259 [00:02<05:33,  1.30s/it]

C:\Users\halle\miniconda3\envs\my-rdkit-env\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\halle\miniconda3\envs\my-rdkit-env\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\halle\miniconda3\envs\my-rdkit-env\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  2%|▏         | 6/259 [00:03<01:45,  2.40it/s]

...

C:\Users\halle\miniconda3\envs\my-rdkit-env\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  7%|▋         | 19/259 [00:03<00:20, 11.71it/s]

C:\Users\halle\miniconda3\envs\my-rdkit-env\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 259/259 [00:26<00:00,  9.87it/s]


,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,34.976402,23.463844,0,0,62.038685,2.194932,4.389864,62.038685,1.240774,4.739337,...,9.679656,85.301740,708.626790,5.288260,15148,52,202.0,205.0,16.444444,12.5
1,36.390615,24.012434,0,0,64.580899,2.194933,4.389866,64.580899,1.24194,4.77843,...,9.710691,87.480775,734.642440,5.323496,17047,54,210.0,213.0,16.944444,13.0
2,37.804829,24.536480,0,0,67.124041,2.194933,4.389866,67.124041,1.243038,4.816053,...,9.740792,89.653577,762.673741,5.296345,19154,56,218.0,221.0,17.444444,13.5
3,39.219042,25.059210,0,0,69.688605,2.194933,4.389866,69.688605,1.244439,4.852311,...,9.770013,91.820569,790.705041,5.271367,21393,58,226.0,229.0,17.944444,14.0
4,40.633256,25.574715,0,0,72.231747,2.194933,4.389867,72.231747,1.245375,4.8873,...,9.798405,93.982129,818.736341,5.248310,23780,60,234.0,237.0,18.444444,14.5


In [45]:
split_csh = data_csh['split_index']
rt_csh = data_csh['retention_time']
df2['split_index'] = split_csh
df2['retention_time'] = rt_csh
df2_train = df2.loc[df2['split_index'] == 1]
df2_train.head()

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,split_index,retention_time
0,34.976402,23.463844,0,0,62.038685,2.194932,4.389864,62.038685,1.240774,4.739337,...,708.626790,5.288260,15148,52,202.0,205.0,16.444444,12.5,1,9.211
1,36.390615,24.012434,0,0,64.580899,2.194933,4.389866,64.580899,1.24194,4.77843,...,734.642440,5.323496,17047,54,210.0,213.0,16.944444,13.0,1,9.235
3,39.219042,25.059210,0,0,69.688605,2.194933,4.389866,69.688605,1.244439,4.852311,...,790.705041,5.271367,21393,58,226.0,229.0,17.944444,14.0,1,10.223
4,40.633256,25.574715,0,0,72.231747,2.194933,4.389867,72.231747,1.245375,4.8873,...,818.736341,5.248310,23780,60,234.0,237.0,18.444444,14.5,1,10.671
5,42.047469,26.069768,0,0,74.775541,2.194933,4.389867,74.775541,1.246259,4.921106,...,844.751991,5.279700,26399,62,242.0,245.0,18.944444,15.0,1,10.713


In [44]:
df2_test = df2.loc[df2['split_index'] == 2]
df2_test.head()

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,split_index,retention_time
2,37.804829,24.536480,0,0,67.124041,2.194933,4.389866,67.124041,1.243038,4.816053,...,762.673741,5.296345,19154,56,218.0,221.0,17.444444,13.500000,2,3.907
7,44.875897,27.051154,0,0,79.880259,2.194933,4.389867,79.880259,1.248129,4.985472,...,896.783291,5.337996,32097,66,258.0,261.0,19.944444,16.000000,2,5.928
13,38.571404,24.025219,0,0,64.520724,2.62105,5.198663,64.520724,1.265112,4.830766,...,700.615832,5.348212,15878,79,252.0,291.0,16.375,11.666667,2,4.488
25,36.368712,24.939087,1,1,61.339288,2.255006,4.510012,61.339288,1.202731,4.768166,...,745.562155,5.691314,15978,54,216.0,220.0,18.708333,12.208333,2,5.666
30,33.770636,23.178123,1,1,59.036181,2.244132,4.488263,59.036181,1.22992,4.702618,...,703.515205,5.766518,13866,51,198.0,202.0,16.645833,11.833333,2,4.763


In [38]:
from autogluon.tabular import TabularDataset, TabularPredictor
label2 = 'retention_time'
print("Summary of rt variable: \n", df2_train[label2].describe())

Summary of rt variable: 
 count    174.000000
mean       3.721977
std        1.817455
min        0.307000
25%        2.295250
50%        3.556000
75%        5.581250
max        8.175000
Name: retention_time, dtype: float64


In [39]:
save_path_csh = 'rt_predict_models_csh'
predictor2 = TabularPredictor(label=label2, path=save_path_csh).fit(df2_train)

Beginning AutoGluon training ...
AutoGluon will save models to "rt_predict_models_csh\"
AutoGluon Version:  0.4.0
Python Version:     3.9.12
Operating System:   Windows
Train Data Rows:    174
Train Data Columns: 1614
Label Column: retention_time
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (8.175, 0.307, 3.72198, 1.81745)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    7285.02 MB
	Train Data (Original)  Memory Usage: 4.68 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually spec

In [41]:
y_test_csh = df2_test[label2]
test_data_nolab_csh = df2_test.drop(columns=[label2])
test_data_nolab_csh.head()

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,split_index
2,37.804829,24.536480,0,0,67.124041,2.194933,4.389866,67.124041,1.243038,4.816053,...,89.653577,762.673741,5.296345,19154,56,218.0,221.0,17.444444,13.500000,2
7,44.875897,27.051154,0,0,79.880259,2.194933,4.389867,79.880259,1.248129,4.985472,...,100.437512,896.783291,5.337996,32097,66,258.0,261.0,19.944444,16.000000,2
13,38.571404,24.025219,0,0,64.520724,2.62105,5.198663,64.520724,1.265112,4.830766,...,103.376384,700.615832,5.348212,15878,79,252.0,291.0,16.375,11.666667,2
25,36.368712,24.939087,1,1,61.339288,2.255006,4.510012,61.339288,1.202731,4.768166,...,87.039148,745.562155,5.691314,15978,54,216.0,220.0,18.708333,12.208333,2
30,33.770636,23.178123,1,1,59.036181,2.244132,4.488263,59.036181,1.22992,4.702618,...,83.455002,703.515205,5.766518,13866,51,198.0,202.0,16.645833,11.833333,2


In [42]:
predictor2 = TabularPredictor.load(save_path_csh)
y_pred_csh = predictor2.predict(test_data_nolab_csh)
print("Predictions:  \n", y_pred_csh)
perf2 = predictor2.evaluate_predictions(y_true=y_test_csh, y_pred=y_pred_csh, auxiliary_metrics=True)

C:\Users\halle\miniconda3\envs\my-rdkit-env\lib\site-packages\autogluon\features\generators\datetime.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_datetime[datetime_feature] = pd.to_datetime(X[datetime_feature], errors='coerce').fillna(self._fillna_map[datetime_feature])
C:\Users\halle\miniconda3\envs\my-rdkit-env\lib\site-packages\autogluon\features\generators\datetime.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_datetime[datetime_feature + '.' + feature] = getattr(X_datetime[datetime_feature].dt, feature).

Predictions:  
 2      3.646656
7      3.482424
13     3.043617
25     3.495660
30     4.416374
41     4.193861
62     3.601072
63     2.758702
74     3.924470
77     3.321847
107    3.298885
110    2.958723
124    3.975637
129    3.965818
162    4.004577
167    3.975600
170    4.234080
178    3.653269
191    2.463006
193    3.345308
Name: retention_time, dtype: float32


In [43]:
df2_modelpf = predictor.leaderboard(df2_test, silent=True)
df2_modelpf.to_csv('model_performance_csh', index=False)

C:\Users\halle\miniconda3\envs\my-rdkit-env\lib\site-packages\autogluon\features\generators\datetime.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_datetime[datetime_feature] = pd.to_datetime(X[datetime_feature], errors='coerce').fillna(self._fillna_map[datetime_feature])
C:\Users\halle\miniconda3\envs\my-rdkit-env\lib\site-packages\autogluon\features\generators\datetime.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_datetime[datetime_feature + '.' + feature] = getattr(X_datetime[datetime_feature].dt, feature).